## Applying Genetic Algorithm for Credit Risk Modeling

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('credit_risk_data.csv')

# Display the first few rows of the dataset
data.head()


,CustomerID,Age,Income,LoanAmount,CreditScore,LoanTerm,DefaultProbability,Defaulted
0,1,56,89240.244639,22969.216211,831.326128,24,0.334193,0
1,2,69,51521.165473,11858.720600,718.570785,36,0.730870,1
2,3,46,55001.620023,49661.757597,822.924989,60,0.420011,0
3,4,32,69046.941204,46715.043557,686.397276,36,0.578360,1
4,5,60,22609.255711,29298.070863,463.928715,24,0.832612,1


In [3]:
from sklearn.model_selection import train_test_split

# Define features and target
features = data[['Age', 'Income', 'LoanAmount', 'CreditScore', 'LoanTerm', 'DefaultProbability']]
target = data['Defaulted']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

print(f'Training samples: {len(X_train)}, Testing samples: {len(X_test)}')


Training samples: 800, Testing samples: 200


In [4]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# Fitness function
def evaluate_model(params, X_train, y_train, X_test, y_test):
    # Define the model with parameters
    model = LogisticRegression(C=params[0], max_iter=int(params[1]), tol=params[2])
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)

    return accuracy

# Initialize population
def initialize_population(pop_size, param_bounds):
    return [np.random.uniform(low, high, size=(len(param_bounds),)) for low, high in param_bounds]

# Selection function
def selection(population, fitness_scores, num_parents):
    parents = [population[idx] for idx in np.argsort(fitness_scores)[-num_parents:]]
    return parents

# Crossover function
def crossover(parents, offspring_size):
    offspring = []
    for _ in range(offspring_size):
        parent1_idx, parent2_idx = np.random.choice(len(parents), size=2, replace=False)
        parent1, parent2 = parents[parent1_idx], parents[parent2_idx]
        crossover_point = np.random.randint(1, len(parent1) - 1)
        child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        offspring.append(child)
    return offspring

# Mutation function
def mutation(offspring, mutation_rate=0.01):
    for individual in offspring:
        if np.random.rand() < mutation_rate:
            mutation_idx = np.random.randint(len(individual))
            individual[mutation_idx] += np.random.uniform(-0.1, 0.1)
    return offspring


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
data = pd.read_csv('credit_risk_data.csv')

# Define features and target
features = data[['Age', 'Income', 'LoanAmount', 'CreditScore', 'LoanTerm', 'DefaultProbability']]
target = data['Defaulted']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f'Training samples: {len(X_train)}, Testing samples: {len(X_test)}')


Training samples: 800, Testing samples: 200


In [10]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# Fitness function
def evaluate_model(params, X_train, y_train, X_test, y_test):
    # Define the model with parameters
    model = LogisticRegression(C=params[0], max_iter=int(params[1]), tol=params[2], solver='lbfgs')
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)

    return accuracy

# Initialize population
def initialize_population(pop_size, param_bounds):
    return [np.random.uniform(low, high, size=(len(param_bounds),)) for low, high in param_bounds]

# Selection function
def selection(population, fitness_scores, num_parents):
    parents = [population[idx] for idx in np.argsort(fitness_scores)[-num_parents:]]
    return parents

# Crossover function
def crossover(parents, offspring_size):
    offspring = []
    for _ in range(offspring_size):
        parent1_idx, parent2_idx = np.random.choice(len(parents), size=2, replace=False)
        parent1, parent2 = parents[parent1_idx], parents[parent2_idx]
        crossover_point = np.random.randint(1, len(parent1) - 1)
        child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        offspring.append(child)
    return offspring

# Mutation function
def mutation(offspring, mutation_rate=0.01):
    for individual in offspring:
        if np.random.rand() < mutation_rate:
            mutation_idx = np.random.randint(len(individual))
            individual[mutation_idx] += np.random.uniform(-0.1, 0.1)
    return offspring


In [11]:
def genetic_algorithm(X_train, y_train, X_test, y_test, num_generations, pop_size, num_parents, mutation_rate):
    param_bounds = [(0.01, 10.0), (100, 500), (0.0001, 0.1)]  # Example bounds for C, max_iter, and tol
    population = initialize_population(pop_size, param_bounds)

    for generation in range(num_generations):
        fitness_scores = [evaluate_model(individual, X_train, y_train, X_test, y_test) for individual in population]
        parents = selection(population, fitness_scores, num_parents)
        offspring_size = pop_size - len(parents)
        offspring = crossover(parents, offspring_size)
        offspring = mutation(offspring, mutation_rate)
        population = parents + offspring

        best_fitness = np.max(fitness_scores)
        print(f"Generation {generation}: Best Fitness = {best_fitness}")

    best_params = population[np.argmax(fitness_scores)]
    return best_params

# Run the genetic algorithm
num_generations = 50
pop_size = 100
num_parents = 20
mutation_rate = 0.01

best_params = genetic_algorithm(X_train, y_train, X_test, y_test, num_generations, pop_size, num_parents, mutation_rate)
print(f"Best Parameters: {best_params}")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Generation 0: Best Fitness = 0.995


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Generation 1: Best Fitness = 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Generation 48: Best Fitness = 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Generation 49: Best Fitness = 1.0
Best Parameters: [0.08696663 5.83287391 6.26504312]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c